# **GUIDE**

In [1]:
import streamlit as st 
import pandas as pd 
import numpy as np

import datetime
from datetime import datetime,timedelta

import plotly
import plotly.express as px 
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## **Requerimientos**

**1 Reporte de impagos** 
* Cantidad y monto
* Impagos acumulados
* Tasa de impago

**2 Reporte de mora**
* Composición de cartera
* Cantidad y monto
* Comportamiento de cliente

In [128]:
mora = pd.read_csv('mora.csv')
cashflow=pd.read_csv('cashflow.csv')

In [3]:
mora.columns

Index(['pais', 'id_credito', 'nombre_cliente', 'documento_cliente',
       'fecha_solicitud', 'fecha_desembolso', 'estado_financiamiento',
       'telefono_cliente', 'telefono2_cliente', 'telefono3_cliente',
       'dias_mora', 'fecha_ultimo_pago', 'monto_ultimo_pago', 'saldo_total',
       'cuotas_totales', 'vendedor', 'nombre_empresa', 'nombre_sucursal',
       'cuota', 'fecha_mora', 'fecha_proxima_cuota', 'exigible_moneda',
       'al_dia', 'mora_1', 'mora_2', 'mora_3', 'mora_4', 'mora_default',
       'cuotas_pendientes'],
      dtype='object')

In [4]:
cashflow.columns

Index(['id_amortizacion', 'id_credito', 'num_cuota', 'saldo_total',
       'monto_cuota', 'saldo_exigible', 'estado', 'fecha_cuota', 'fecha_pago',
       'monto_seguro', 'pais', 'estatus', 'cuota_moneda', 'exigible_moneda',
       'tasa', 'numero_periodos', 'score_hit', 'score_no_hit',
       'valor_financiamiento', 'prima', 'accesorios'],
      dtype='object')

**Variables constantes por id_credito**
 
 * fecha_solicitud --> cosecha cohort
 * pais
 * empresa
 * sucursal
 * vendedor 
 * cuota
 * num_cuotas

**Variables actuales por id_credito**

* num_cuotas
* cuotas_pendientes
* tasa_impago
* categoría mora
* pagado
* exigible
* fijo

In [5]:
today = datetime.today()

In [118]:
cashflow.sample(10)

,id_amortizacion,id_credito,num_cuota,saldo_total,monto_cuota,saldo_exigible,estado,fecha_cuota,fecha_pago,monto_seguro,...,exigible_moneda,tasa,numero_periodos,score_hit,score_no_hit,valor_financiamiento,prima,accesorios,cuotas_pendientes,mora
8816,9378,SV0000245,4,166.16,33.23,NaN,Fijo,2024-10-30,NaN,NaN,...,NaN,4.15,8,0,480,133.6,33.4,0.0,1.0,Al día
14394,15028,HN0001040,6,213.36,30.48,NaN,Fijo,2024-12-05,NaN,NaN,...,NaN,3.50,12,0,711,4100.0,900.0,500.0,1.0,Al día
25935,26793,HN0001775,11,199.13,33.24,NaN,Fijo,2025-03-11,NaN,NaN,...,NaN,3.50,16,0,707,4960.0,1240.0,0.0,NaN,Al día
14217,14851,HN0001026,9,222.65,27.83,NaN,Fijo,2025-01-19,NaN,NaN,...,NaN,4.15,16,0,458,3650.0,1350.0,500.0,1.0,Al día
3065,3499,SV0000101,17,136.22,34.05,NaN,Fijo,2025-05-02,NaN,NaN,...,NaN,4.00,20,0,662,192.8,48.2,0.0,0.0,Al día
2324,2726,SV0000085,20,21.26,21.26,NaN,Fijo,2025-06-15,NaN,NaN,...,NaN,4.00,20,0,662,120.0,30.0,0.0,0.0,Al día
1678,2048,HN0000208,6,364.86,33.17,NaN,Fijo,2024-11-15,NaN,NaN,...,NaN,4.15,16,0,563,4350.0,1150.0,0.0,3.0,Al día
1359,1689,SV0000058,3,257.72,42.95,NaN,Pagado,2024-09-29,2024-10-01,NaN,...,NaN,3.20,8,0,769,197.0,110.0,0.0,NaN,Pagado
7147,7697,HN0000592,3,126.73,21.12,NaN,Fijo,2024-10-12,NaN,NaN,...,NaN,3.50,8,0,713,2320.0,580.0,0.0,2.0,Al día
25416,26258,SV0000710,4,149.60,16.00,NaN,Fijo,2024-11-25,NaN,NaN,...,NaN,3.20,12,0,814,93.6,23.4,0.0,NaN,Al día


In [129]:
cashflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29349 entries, 0 to 29348
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id_amortizacion       29349 non-null  int64  
 1   id_credito            29349 non-null  object 
 2   num_cuota             29349 non-null  int64  
 3   saldo_total           29349 non-null  float64
 4   monto_cuota           29349 non-null  float64
 5   saldo_exigible        1431 non-null   float64
 6   estado                29349 non-null  object 
 7   fecha_cuota           29349 non-null  object 
 8   fecha_pago            1720 non-null   object 
 9   monto_seguro          240 non-null    float64
 10  pais                  29349 non-null  object 
 11  estatus               0 non-null      float64
 12  cuota_moneda          29349 non-null  float64
 13  exigible_moneda       1431 non-null   float64
 14  tasa                  29349 non-null  float64
 15  numero_periodos    

In [130]:
cashflow.select_dtypes('object')

,id_credito,estado,fecha_cuota,fecha_pago,pais
0,SV0000006,Vencido,2024-08-18,NaN,El Salvador
1,SV0000009,Pagado Retraso,2024-08-18,2024-08-24,El Salvador
2,HN0000010,Pagado a Tiempo,2024-08-22,2024-08-22,Honduras
3,HN0000011,Pagado Retraso,2024-08-21,2024-08-24,Honduras
4,SV0000021,Pagado Retraso,2024-08-21,2024-08-22,El Salvador
...,...,...,...,...,...
29344,HN0001978,Fijo,2025-04-01,NaN,Honduras
29345,HN0001978,Fijo,2025-04-16,NaN,Honduras
29346,HN0001978,Fijo,2025-05-01,NaN,Honduras
29347,HN0001978,Fijo,2025-05-16,NaN,Honduras


**To do!** 

* Cambiar estado
* Procesar columnas de fecha como datetime y cohort

In [145]:
cashflow['estado_pago'] = cashflow['estado'].map({'Pagado a Tiempo':'Pagado',
                                                  'Pagado Retraso':'Pagado',
                                                  'Vencido':'Impago',
                                                  'Exigible':'Al día',
                                                  'Fijo':'Al día'})

In [133]:
cashflow['fecha_cuota'] = pd.to_datetime(cashflow['fecha_cuota'])
cashflow['cohort_D'] = cashflow['fecha_cuota'].astype(str)

In [131]:
cashflow.select_dtypes(['int','float'])

,id_amortizacion,num_cuota,saldo_total,monto_cuota,saldo_exigible,monto_seguro,estatus,cuota_moneda,exigible_moneda,tasa,numero_periodos,score_hit,score_no_hit,valor_financiamiento,prima,accesorios
0,3,1,186.71,23.34,23.34,NaN,NaN,23.34,23.34,4.30,8,0,427,92.0,23.0,0.0
1,4,1,170.48,21.31,NaN,NaN,NaN,21.31,NaN,4.30,8,0,388,84.0,21.0,0.0
2,6,1,392.01,32.67,NaN,NaN,NaN,816.69,NaN,4.15,12,0,489,3928.2,919.8,249.0
3,12,1,325.36,27.11,NaN,NaN,NaN,677.84,NaN,3.50,12,0,707,3647.2,799.8,448.0
4,14,1,163.84,20.48,NaN,NaN,NaN,20.48,NaN,4.00,8,0,663,84.0,21.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29344,30234,12,127.89,25.64,NaN,NaN,NaN,641.00,NaN,4.15,16,0,637,3360.0,1440.0,0.0
29345,30235,13,102.25,25.64,NaN,NaN,NaN,641.00,NaN,4.15,16,0,637,3360.0,1440.0,0.0
29346,30236,14,76.61,25.64,NaN,NaN,NaN,641.00,NaN,4.15,16,0,637,3360.0,1440.0,0.0
29347,30237,15,50.97,25.64,NaN,NaN,NaN,641.00,NaN,4.15,16,0,637,3360.0,1440.0,0.0


**To do!**

* Num_cuota como categoría, no número
* monto_cuota como valor target

In [171]:
cashflow['edad_quincenas'] = cashflow['num_cuota'].astype(str)

In [140]:
cashflow.select_dtypes('object')

,id_credito,num_cuota,estado,fecha_pago,pais,estado_pago,cohort_D
0,SV0000006,1,Vencido,NaN,El Salvador,Impago,2024-08-18
1,SV0000009,1,Pagado Retraso,2024-08-24,El Salvador,Pagado,2024-08-18
2,HN0000010,1,Pagado a Tiempo,2024-08-22,Honduras,Pagado,2024-08-22
3,HN0000011,1,Pagado Retraso,2024-08-24,Honduras,Pagado,2024-08-21
4,SV0000021,1,Pagado Retraso,2024-08-22,El Salvador,Pagado,2024-08-21
...,...,...,...,...,...,...,...
29344,HN0001978,12,Fijo,NaN,Honduras,Al día,2025-04-01
29345,HN0001978,13,Fijo,NaN,Honduras,Al día,2025-04-16
29346,HN0001978,14,Fijo,NaN,Honduras,Al día,2025-05-01
29347,HN0001978,15,Fijo,NaN,Honduras,Al día,2025-05-16


In [172]:
cashflow.groupby(['pais','estado_pago','id_credito','edad_quincenas','num_cuota','fecha_cuota'])[['id_amortizacion','monto_cuota']].agg({'id_amortizacion':'count',
                                                                                                                   'monto_cuota':'sum'}).sort_index(level=[3,4])

id_amortizacion  \
pais        estado_pago id_credito edad_quincenas num_cuota fecha_cuota                    
El Salvador Al día      SV0000580  1              1         2024-10-03                 1   
                        SV0000582  1              1         2024-10-03                 1   
                        SV0000584  1              1         2024-10-03                 1   
                        SV0000585  1              1         2024-10-03                 1   
                        SV0000586  1              1         2024-10-03                 1   
...                                                                                  ...   
Honduras    Al día      HN0001969  9              9         2025-02-14                 1   
                        HN0001970  9              9         2025-02-14                 1   
                        HN0001973  9              9         2025-02-15                 1   
                        HN0001977  9              9         2025-02-15                 1   
                        HN0001978  9              9         2025-02-15                 1   

                                                                         monto_cuota  
pais        estado_pago id_credito edad_quincenas num_cuota fecha_cuota               
El Salvador Al día      SV0000580  1              1         2024-10-03         32.57  
                        SV0000582  1              1         2024-10-03         23.67  
                        SV0000584  1              1         2024-10-03         42.04  
                        SV0000585  1              1         2024-10-03         29.13  
                        SV0000586  1              1         2024-10-03         31.57  
...                                                                              ...  
Honduras    Al día      HN0001969  9              9         2025-02-14         30.36  
                        HN0001970  9              9         2025-02-14         23.28  
                        HN0001973  9              9         2025-02-15         23.28  
                        HN0001977  9              9         2025-02-15         24.36  
                        HN0001978  9              9         2025-02-15         25.64  

[29349 rows x 2 columns]

In [183]:
grouped = cashflow.groupby(['pais','estado_pago','id_credito','edad_quincenas','num_cuota','fecha_cuota'])[['id_amortizacion','monto_cuota']].agg({'id_amortizacion':'count',
                                                                                                                   'monto_cuota':'sum'}).sort_index(level=[3,4]).reset_index()

In [184]:
grouped['num_cuota'] = grouped['num_cuota'].astype(int)

In [198]:
grouped.sample(15)

,pais,estado_pago,id_credito,edad_quincenas,num_cuota,fecha_cuota,id_amortizacion,monto_cuota,recibido,monto_recibido,pagos_esperados,recibido_acumulado
13747,Honduras,Al día,HN0000702,3,3,2024-10-15,1,13.87,NaN,NaN,10,NaN
131,El Salvador,Al día,SV0000723,1,1,2024-10-12,1,19.00,NaN,NaN,1,NaN
10871,Honduras,Al día,HN0001279,2,2,2024-10-11,1,18.68,NaN,NaN,9,NaN
14850,Honduras,Al día,HN0001849,3,3,2024-11-14,1,34.84,NaN,NaN,3,NaN
314,El Salvador,Impago,SV0000390,1,1,2024-09-22,1,19.12,0.0,0.0,1,0.0
28029,Honduras,Al día,HN0001924,8,8,2025-01-29,1,22.16,NaN,NaN,11,NaN
9,El Salvador,Al día,SV0000593,1,1,2024-10-03,1,19.46,NaN,NaN,1,NaN
9013,Honduras,Al día,HN0000783,16,16,2025-04-30,1,27.83,NaN,NaN,8,NaN
23332,El Salvador,Al día,SV0000482,7,7,2024-12-26,1,39.70,NaN,NaN,10,NaN
8456,Honduras,Al día,HN0001190,15,15,2025-04-22,1,15.48,NaN,NaN,7,NaN


In [209]:
grouped[grouped['id_credito']=='SV0000390']

,pais,estado_pago,id_credito,edad_quincenas,num_cuota,fecha_cuota,id_amortizacion,monto_cuota,recibido,monto_recibido,pagos_esperados,recibido_acumulado
314,El Salvador,Impago,SV0000390,1,1,2024-09-22,1,19.12,0.0,0.0,1,0.0
9697,El Salvador,Al día,SV0000390,2,2,2024-10-07,1,19.12,NaN,NaN,2,NaN
12848,El Salvador,Al día,SV0000390,3,3,2024-10-22,1,19.12,NaN,NaN,3,NaN
15477,El Salvador,Al día,SV0000390,4,4,2024-11-06,1,19.12,NaN,NaN,4,NaN
18067,El Salvador,Al día,SV0000390,5,5,2024-11-21,1,19.12,NaN,NaN,5,NaN
20654,El Salvador,Al día,SV0000390,6,6,2024-12-06,1,19.12,NaN,NaN,6,NaN
23241,El Salvador,Al día,SV0000390,7,7,2024-12-21,1,19.12,NaN,NaN,7,NaN
25828,El Salvador,Al día,SV0000390,8,8,2025-01-05,1,19.11,NaN,NaN,8,NaN


In [185]:
grouped['recibido'] = grouped['estado_pago'].map({'Pagado':1,
                                                  'Impago':0})

In [186]:
grouped['monto_recibido'] = grouped['recibido'] * grouped['monto_cuota']

In [199]:
grouped.sort_values(by=['id_credito','num_cuota'],inplace=True)

In [203]:
grouped['pagos_esperados'] = grouped.groupby(['id_credito'])['id_amortizacion'].cumsum()

In [207]:
grouped['recibido_acumulado'] = grouped.groupby('id_credito')['recibido'].cumsum()

In [214]:
grouped['cuotas_pendientes'] = (grouped['pagos_esperados'] - grouped['recibido_acumulado'].fillna(0)).astype(int)
grouped['mora'] = 'Mora '+grouped['cuotas_pendientes'].astype(str)

In [216]:
grouped['mora'] = grouped['mora'].where(grouped['estado_pago']=='Impago','Al día')

In [217]:
grouped['mora'] = grouped['mora'].where(grouped['estado_pago']!='Pagado','Pagado')

In [227]:
grouped

,pais,estado_pago,id_credito,edad_quincenas,num_cuota,fecha_cuota,id_amortizacion,monto_cuota,recibido,monto_recibido,pagos_esperados,recibido_acumulado,cuotas_pendientes,mora,monto_esperado,monto_recibido_acumulado
1889,Honduras,Pagado,HN0000010,1.0,1,2024-08-22,1,32.67,1.0,32.67,1,1.0,0,Pagado,32.67,32.67
11929,Honduras,Pagado,HN0000010,2.0,2,2024-09-04,1,32.67,1.0,32.67,2,2.0,0,Pagado,65.34,65.34
15068,Honduras,Pagado,HN0000010,3.0,3,2024-09-19,1,32.67,1.0,32.67,3,3.0,0,Pagado,98.01,98.01
15926,Honduras,Al día,HN0000010,NaN,4,2024-10-04,1,32.67,NaN,NaN,4,NaN,4,Al día,130.68,NaN
18513,Honduras,Al día,HN0000010,NaN,5,2024-10-19,1,32.67,NaN,NaN,5,NaN,5,Al día,163.35,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26273,El Salvador,Al día,SV0000846,NaN,8,2025-01-31,1,62.00,NaN,NaN,8,NaN,8,Al día,496.00,NaN
28431,El Salvador,Al día,SV0000846,NaN,9,2025-02-15,1,62.00,NaN,NaN,9,NaN,9,Al día,558.00,NaN
2938,El Salvador,Al día,SV0000846,NaN,10,2025-03-02,1,62.00,NaN,NaN,10,NaN,10,Al día,620.00,NaN
4206,El Salvador,Al día,SV0000846,NaN,11,2025-03-17,1,62.00,NaN,NaN,11,NaN,11,Al día,682.00,NaN


In [226]:
grouped['edad_quincenas'] = grouped['edad_quincenas'].astype(int).where(grouped['estado_pago']!='Al día',np.nan)

In [ ]:
grouped['edad_quincenas']

In [219]:
grouped['monto_esperado'] = grouped.groupby('id_credito')['monto_cuota'].cumsum()

In [221]:
grouped['monto_recibido_acumulado'] = grouped.groupby('id_credito')['monto_recibido'].cumsum()

In [ ]:
# Ver por: fecha, cosecha, estado_mora, país, pequeñas categorías (ranking)
# Ver: cantidad, monto
# Ver: total, relativo, acumulado, acumulado relativo

# Comparar: país, mora 

# Comparar nivel 1 color, 2 facet_col, 3 facet_row

In [240]:
category = 'fecha_cuota'
modo = 'sum'
variable = 'monto_cuota'
compare = 'mora'

In [252]:
data = grouped.merge(mora[['id_credito','fecha_solicitud','nombre_empresa','nombre_sucursal','vendedor']], how='left', on='id_credito')
data.sample(10)

,pais,estado_pago,id_credito,edad_quincenas,num_cuota,fecha_cuota,id_amortizacion,monto_cuota,recibido,monto_recibido,pagos_esperados,recibido_acumulado,cuotas_pendientes,mora,monto_esperado,monto_recibido_acumulado,fecha_solicitud,nombre_empresa,nombre_sucursal,vendedor
10915,Honduras,Al día,HN0001031,NaN,7,2024-12-20,1,26.38,NaN,NaN,7,NaN,7,Al día,184.66,NaN,2024-09-06,GALO CELL,Tienda principal- B,Celio Yobany Lobo Erazo
20065,Honduras,Al día,HN0001926,NaN,5,2024-12-15,1,31.36,NaN,NaN,5,NaN,5,Al día,156.80,NaN,NaN,NaN,NaN,NaN
8352,Honduras,Al día,HN0000821,NaN,4,2024-11-02,1,32.03,NaN,NaN,4,NaN,4,Al día,128.12,NaN,2024-09-03,(ANTERIOR) INVERSIONES EBEN EZER,TIENDA CARPA,Lilian Oneyda López Lorenzo
6779,Honduras,Al día,HN0000687,NaN,3,2024-10-15,1,26.38,NaN,NaN,3,NaN,3,Al día,79.14,NaN,NaN,NaN,NaN,NaN
28553,El Salvador,Al día,SV0000781,NaN,1,2024-10-15,1,41.00,NaN,NaN,1,NaN,1,Al día,41.00,NaN,NaN,NaN,NaN,NaN
23471,El Salvador,Al día,SV0000321,NaN,7,2024-12-17,1,18.92,NaN,NaN,7,NaN,7,Al día,132.44,NaN,NaN,NaN,NaN,NaN
25875,El Salvador,Al día,SV0000535,NaN,7,2024-12-29,1,36.15,NaN,NaN,7,NaN,7,Al día,253.05,NaN,2024-09-15,"Maverick El Salvador, S.A. de C.V.",Tienda Maverick San Miguel #3,ANGELICA KARINA MELGAR LAZO
983,Honduras,Impago,HN0000163,3.0,3,2024-09-29,1,24.40,0.0,0.0,3,0.0,3,Mora 3,73.20,0.0,2024-08-15,GALO CELL,Tienda principal- B,jasser steven valeriano padilla
10514,Honduras,Al día,HN0000992,NaN,10,2025-02-02,1,27.83,NaN,NaN,10,NaN,10,Al día,278.30,NaN,2024-09-05,(ANTERIOR) INVERSIONES EBEN EZER,TIENDA CARPA,LilianTurcios
9502,Honduras,Al día,HN0000908,NaN,14,2025-04-02,1,27.83,NaN,NaN,14,NaN,14,Al día,389.62,NaN,2024-09-04,(ANTERIOR) INVERSIONES EBEN EZER,TIENDA CARPA,Lilian Oneyda López Lorenzo


In [274]:
def prep(cashflow, mora):

    mora = pd.read_csv(mora)
    cashflow=pd.read_csv(cashflow)

    cashflow['fecha_cuota'] = pd.to_datetime(cashflow['fecha_cuota'])
    mora['fecha_solicitud'] = pd.to_datetime(mora['fecha_solicitud'])

    cashflow['estado_pago'] = cashflow['estado'].map({'Pagado a Tiempo':'Pagado',
                                                  'Pagado Retraso':'Pagado',
                                                  'Vencido':'Impago',
                                                  'Exigible':'Al día',
                                                  'Fijo':'Al día'})

    grouped = cashflow.groupby(['pais','estado_pago','id_credito','num_cuota','fecha_cuota'])[['id_amortizacion','monto_cuota']].agg({'id_amortizacion':'count',
                                                                                                                   'monto_cuota':'sum'}).sort_index(level=[3,4]).reset_index()

    grouped['recibido'] = grouped['estado_pago'].map({'Pagado':1,
                                                  'Impago':0})
    grouped['monto_recibido'] = grouped['recibido'] * grouped['monto_cuota']

    grouped.sort_values(by=['id_credito','num_cuota'],inplace=True)
    grouped['monto_esperado'] = grouped.groupby('id_credito')['monto_cuota'].cumsum()
    grouped['monto_recibido_acumulado'] = grouped.groupby('id_credito')['monto_recibido'].cumsum()
    grouped['pagos_esperados'] = grouped.groupby(['id_credito'])['id_amortizacion'].cumsum()
    grouped['recibido_acumulado'] = grouped.groupby('id_credito')['recibido'].cumsum()
    grouped['cuotas_pendientes'] = (grouped['pagos_esperados'] - grouped['recibido_acumulado'].fillna(0)).astype(int)
    grouped['mora'] = 'Mora '+grouped['cuotas_pendientes'].astype(str)
    grouped['mora'] = grouped['mora'].where(grouped['estado_pago']=='Impago','Al día')
    grouped['mora'] = grouped['mora'].where(grouped['estado_pago']!='Pagado','Pagado')

    data = grouped.merge(mora[['id_credito','fecha_solicitud','nombre_empresa','nombre_sucursal','vendedor']], how='left', on='id_credito')

    return data

In [275]:
prep('cashflow.csv','mora.csv')

,pais,estado_pago,id_credito,num_cuota,fecha_cuota,id_amortizacion,monto_cuota,recibido,monto_recibido,monto_esperado,monto_recibido_acumulado,pagos_esperados,recibido_acumulado,cuotas_pendientes,mora,fecha_solicitud,nombre_empresa,nombre_sucursal,vendedor
0,Honduras,Pagado,HN0000010,1,2024-08-22,1,32.67,1.0,32.67,32.67,32.67,1,1.0,0,Pagado,2024-08-05,IMPORTADORA GODOY S de RL,Mall Premier-1,Amarily Quiroz
1,Honduras,Pagado,HN0000010,2,2024-09-04,1,32.67,1.0,32.67,65.34,65.34,2,2.0,0,Pagado,2024-08-05,IMPORTADORA GODOY S de RL,Mall Premier-1,Amarily Quiroz
2,Honduras,Pagado,HN0000010,3,2024-09-19,1,32.67,1.0,32.67,98.01,98.01,3,3.0,0,Pagado,2024-08-05,IMPORTADORA GODOY S de RL,Mall Premier-1,Amarily Quiroz
3,Honduras,Al día,HN0000010,4,2024-10-04,1,32.67,NaN,NaN,130.68,NaN,4,NaN,4,Al día,2024-08-05,IMPORTADORA GODOY S de RL,Mall Premier-1,Amarily Quiroz
4,Honduras,Al día,HN0000010,5,2024-10-19,1,32.67,NaN,NaN,163.35,NaN,5,NaN,5,Al día,2024-08-05,IMPORTADORA GODOY S de RL,Mall Premier-1,Amarily Quiroz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29344,El Salvador,Al día,SV0000846,8,2025-01-31,1,62.00,NaN,NaN,496.00,NaN,8,NaN,8,Al día,NaT,NaN,NaN,NaN
29345,El Salvador,Al día,SV0000846,9,2025-02-15,1,62.00,NaN,NaN,558.00,NaN,9,NaN,9,Al día,NaT,NaN,NaN,NaN
29346,El Salvador,Al día,SV0000846,10,2025-03-02,1,62.00,NaN,NaN,620.00,NaN,10,NaN,10,Al día,NaT,NaN,NaN,NaN
29347,El Salvador,Al día,SV0000846,11,2025-03-17,1,62.00,NaN,NaN,682.00,NaN,11,NaN,11,Al día,NaT,NaN,NaN,NaN


In [276]:
data.mora.unique()

array(['Pagado', 'Al día', 'Mora 1', 'Mora 2', 'Mora 3', 'Mora 4'],
      dtype=object)

In [279]:
data['semana'] = pd.to_datetime(data['fecha_solicitud']).dt.isocalendar().week

In [289]:
data.groupby(['semana','vendedor'])[['id_credito']].count()

id_credito
semana vendedor                                        
31     Eduarda del Carmen Contreras Cosme             8
       Jenmy Katherine Herrera De Aguilar             8
32     ARNOLD STEVEN ALVAREZ LAZO                    12
       Alba Hernandez                                12
       Alejandra Matute                              20
...                                                 ...
39     WILBERT RENE VENTURA PEREZ                     8
       Xamira Yanka Suri Guzmán                      12
       Xiomara Jackeline Godoy Molina                28
       Yolanda Marisel Hernández Chinchilla          12
       Yolany Melisa  Argueta Claros                 16

[544 rows x 1 columns]

In [287]:
data.semana.min()

31

In [288]:
data.semana.max()

39

In [294]:
for i in range(data.semana.min(), data.semana.max()+1):
    print(i)

31
32
33
34
35
36
37
38
39


In [293]:
list(range(31,40))

[31, 32, 33, 34, 35, 36, 37, 38, 39]

In [247]:
fig = px.histogram(grouped, template='simple_white', histfunc=modo, x=category, y=variable, color=compare)
fig.show()

In [28]:
cashflow['cuotas_pendientes'] = cashflow['id_credito'].map(mora.set_index('id_credito')['cuotas_pendientes'])

In [32]:
cashflow['mora'] = None

for item in cashflow.index:
    if cashflow.at[item, 'estado'] == 'Vencido':
        cashflow.at[item, 'mora'] = 'Mora '+str(round(cashflow.at[item, 'cuotas_pendientes']))
    elif cashflow.at[item, 'estado'] == 'Pagado':
        cashflow.at[item,'mora'] = 'Pagado'
    else:
        cashflow.at[item, 'mora'] = 'Al día'

In [33]:
cashflow

,id_amortizacion,id_credito,num_cuota,saldo_total,monto_cuota,saldo_exigible,estado,fecha_cuota,fecha_pago,monto_seguro,...,exigible_moneda,tasa,numero_periodos,score_hit,score_no_hit,valor_financiamiento,prima,accesorios,cuotas_pendientes,mora
0,3,SV0000006,1,186.71,23.34,23.34,Vencido,2024-08-18,NaN,NaN,...,23.34,4.30,8,0,427,92.0,23.0,0.0,4.0,Mora 4
1,4,SV0000009,1,170.48,21.31,NaN,Pagado,2024-08-18,2024-08-24,NaN,...,NaN,4.30,8,0,388,84.0,21.0,0.0,2.0,Pagado
2,6,HN0000010,1,392.01,32.67,NaN,Pagado,2024-08-22,2024-08-22,NaN,...,NaN,4.15,12,0,489,3928.2,919.8,249.0,0.0,Pagado
3,12,HN0000011,1,325.36,27.11,NaN,Pagado,2024-08-21,2024-08-24,NaN,...,NaN,3.50,12,0,707,3647.2,799.8,448.0,0.0,Pagado
4,14,SV0000021,1,163.84,20.48,NaN,Pagado,2024-08-21,2024-08-22,NaN,...,NaN,4.00,8,0,663,84.0,21.0,0.0,0.0,Pagado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29344,30234,HN0001978,12,127.89,25.64,NaN,Fijo,2025-04-01,NaN,NaN,...,NaN,4.15,16,0,637,3360.0,1440.0,0.0,NaN,Al día
29345,30235,HN0001978,13,102.25,25.64,NaN,Fijo,2025-04-16,NaN,NaN,...,NaN,4.15,16,0,637,3360.0,1440.0,0.0,NaN,Al día
29346,30236,HN0001978,14,76.61,25.64,NaN,Fijo,2025-05-01,NaN,NaN,...,NaN,4.15,16,0,637,3360.0,1440.0,0.0,NaN,Al día
29347,30237,HN0001978,15,50.97,25.64,NaN,Fijo,2025-05-16,NaN,NaN,...,NaN,4.15,16,0,637,3360.0,1440.0,0.0,NaN,Al día


In [6]:
# Columna cosecha cohort
mora['fecha_solicitud'] = pd.to_datetime(mora['fecha_solicitud'])

mora['edad_D'] = [(today - date).days for date in mora['fecha_solicitud']]
mora['edad_W'] = (mora['edad_D']/7).apply(np.floor).astype(int)
mora['edad_Q'] = (mora['edad_D']/15).apply(np.floor).astype(int)

In [18]:
# Extraer estado actual
mora[mora['cuotas_pendientes']==1]

,pais,id_credito,nombre_cliente,documento_cliente,fecha_solicitud,fecha_desembolso,estado_financiamiento,telefono_cliente,telefono2_cliente,telefono3_cliente,...,mora_1,mora_2,mora_3,mora_4,mora_default,cuotas_pendientes,w_cohort,edad_D,edad_W,edad_Q
2,El Salvador,SV0000067,ELIANA MELISSA HERNANDEZ GRANDE,03767135-3,2024-08-17,NaN,NaN,50379261549,503,503,...,19.51,0.0,0.0,0.0,0.0,1,33,58,8,3
11,El Salvador,SV0000281,NELSON ARMANDO AMAYA AMAYA,06514836-8,2024-09-01,NaN,NaN,50369632239,503,503,...,26.86,0.0,0.0,0.0,0.0,1,35,43,6,2
14,El Salvador,SV0000356,KAREN ALEJANDRA SA HERRERA RAMOS,05916487-7,2024-09-05,NaN,NaN,50373373410,503,503,...,29.13,0.0,0.0,0.0,0.0,1,36,39,5,2
15,El Salvador,SV0000390,WILLIAM ARMANDO RAMIREZ ROSALES,02705506-8,2024-09-07,NaN,NaN,50360086557,50376157971,503,...,19.12,0.0,0.0,0.0,0.0,1,36,37,5,2
25,El Salvador,SV0000236,MARIA MAGDALENA PINEDA AQUINO,04774171-0,2024-08-31,NaN,NaN,50360526787,503,503,...,27.94,0.0,0.0,0.0,0.0,1,35,44,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1606,Honduras,HN0001347,JOSE MIGUEL ARTURO CRUZ MAYA,0418-1967-00040,2024-09-14,NaN,NaN,50494056509,504,504,...,528.03,0.0,0.0,0.0,0.0,1,37,30,4,2
1607,Honduras,HN0001388,PAPSI FABIOLA AVILA PONCE,0801-1990-13185,2024-09-15,NaN,NaN,50489677741,504,504,...,883.59,0.0,0.0,0.0,0.0,1,37,29,4,1
1608,Honduras,HN0001414,ELDER noe PADILLA padilla,1508-1990-00185,2024-09-16,NaN,NaN,50492762153,504,504,...,783.51,0.0,0.0,0.0,0.0,1,38,28,4,1
1609,Honduras,HN0001436,MARLON JOSE OCAMPO LEMUS,1801-1998-01715,2024-09-16,NaN,NaN,50432459433,504,504,...,611.24,0.0,0.0,0.0,0.0,1,38,28,4,1


In [19]:
cashflow[cashflow['id_credito']=='SV0000236'] # Monto esperado hasta la fecha es sum(monto_cuota) cuando no es fijo, esperado en el futuro, suma de fijo, vencido es impago

,id_amortizacion,id_credito,num_cuota,saldo_total,monto_cuota,saldo_exigible,estado,fecha_cuota,fecha_pago,monto_seguro,...,estatus,cuota_moneda,exigible_moneda,tasa,numero_periodos,score_hit,score_no_hit,valor_financiamiento,prima,accesorios
8569,9119,SV0000236,1,335.31,27.94,NaN,Pagado a Tiempo,2024-09-15,2024-09-14,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8570,9120,SV0000236,2,307.37,27.94,27.94,Vencido,2024-09-30,NaN,NaN,...,NaN,27.94,27.94,4.15,12,0,501,134.4,33.6,0.0
8571,9121,SV0000236,3,279.43,27.94,NaN,Fijo,2024-10-15,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8572,9122,SV0000236,4,251.49,27.94,NaN,Fijo,2024-10-30,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8573,9123,SV0000236,5,223.55,27.94,NaN,Fijo,2024-11-14,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8574,9124,SV0000236,6,195.61,27.94,NaN,Fijo,2024-11-29,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8575,9125,SV0000236,7,167.67,27.94,NaN,Fijo,2024-12-14,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8576,9126,SV0000236,8,139.73,27.94,NaN,Fijo,2024-12-29,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8577,9127,SV0000236,9,111.79,27.94,NaN,Fijo,2025-01-13,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0
8578,9128,SV0000236,10,83.85,27.94,NaN,Fijo,2025-01-28,NaN,NaN,...,NaN,27.94,NaN,4.15,12,0,501,134.4,33.6,0.0


In [7]:
cashflow['estado'].replace({'Pagado a Tiempo':'Pagado',
                            'Pagado Retraso':'Pagado'}, inplace=True) #PONER AL INICIO

In [34]:
df = cashflow.groupby(['id_credito','mora','estado'])[['fecha_cuota','fecha_pago','monto_cuota','saldo_exigible']].agg({'fecha_cuota':'count','fecha_pago':'count','monto_cuota':'sum','saldo_exigible':'sum'}).reset_index()

In [35]:
df = df.merge(mora, how='left', on='id_credito')

In [36]:
df.columns

Index(['id_credito', 'mora', 'estado', 'fecha_cuota', 'fecha_pago',
       'monto_cuota', 'saldo_exigible', 'pais', 'nombre_cliente',
       'documento_cliente', 'fecha_solicitud', 'fecha_desembolso',
       'estado_financiamiento', 'telefono_cliente', 'telefono2_cliente',
       'telefono3_cliente', 'dias_mora', 'fecha_ultimo_pago',
       'monto_ultimo_pago', 'saldo_total', 'cuotas_totales', 'vendedor',
       'nombre_empresa', 'nombre_sucursal', 'cuota', 'fecha_mora',
       'fecha_proxima_cuota', 'exigible_moneda', 'al_dia', 'mora_1', 'mora_2',
       'mora_3', 'mora_4', 'mora_default', 'cuotas_pendientes', 'edad_D',
       'edad_W', 'edad_Q'],
      dtype='object')

In [37]:
df.drop(columns=['nombre_cliente','documento_cliente','fecha_desembolso','telefono_cliente','telefono2_cliente','telefono3_cliente','exigible_moneda'])

,id_credito,mora,estado,fecha_cuota,fecha_pago,monto_cuota,saldo_exigible,pais,fecha_solicitud,estado_financiamiento,...,al_dia,mora_1,mora_2,mora_3,mora_4,mora_default,cuotas_pendientes,edad_D,edad_W,edad_Q
0,HN0000010,Al día,Fijo,9,0,294.03,0.0,Honduras,2024-08-05,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,10.0,4.0
1,HN0000010,Pagado,Pagado,3,3,98.01,0.0,Honduras,2024-08-05,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,10.0,4.0
2,HN0000011,Al día,Fijo,9,0,243.99,0.0,Honduras,2024-08-06,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,10.0,4.0
3,HN0000011,Pagado,Pagado,3,3,81.33,0.0,Honduras,2024-08-06,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,10.0,4.0
4,HN0000012,Al día,Fijo,5,0,129.35,0.0,Honduras,2024-08-07,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69.0,9.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4742,SV0000842,Al día,Fijo,12,0,619.83,0.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4743,SV0000843,Al día,Fijo,12,0,518.50,0.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4744,SV0000844,Al día,Fijo,8,0,213.31,0.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4745,SV0000845,Al día,Fijo,8,0,233.03,0.0,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
len(cashflow['id_credito'].unique())

2588

In [39]:
len(df['id_credito'].unique())

2588

### **Opciones**

**1 Niveles**


Categorías constantes por crédito.

A. Geográfico: PAIS - DEPARTAMENTO - MUNICIPIO
B. Vendedor: EMPRESA - SUCURSAL - VENDEDOR

Categoría constante: Cosecha

C. Temporal: QUINCENA - SEMANA - DIA

**2 Categorías**

A. Estado

B. Mora

**Variables target:** CANTIDAD Y MONTO 


### **Filtros**

### **Métricas**

Asumiendo 'mora.csv' y 'cashflow.csv'

#### **Totales**

Streamlit metric cards.

In [11]:
# Clientes únicos
clientes_unicos = len(cashflow['id_credito'].unique())

In [12]:
clientes_unicos # TURN INTO CARD

2588

In [13]:
# Venta total monto
df['monto_cuota'].sum()

# % fijo, pagado, impago


824175.1599999999

In [50]:
df['quincena'] = df['edad_Q'].fillna(0).astype(int).astype(str)

In [112]:
fig = px.histogram(df[df['pais'].isin(['El Salvador', 'Honduras'])], template='simple_white', x='quincena', y='monto_cuota', color = 'mora', facet_col='pais', barnorm='percent',
                   color_discrete_map={'Mora 4':'black','Mora 3':'purple','Mora 2':'purple','Mora 1':'violet', 'Pagado':'green','Al día':'lightblue'},
                   category_orders={'quincena':['4','3','2','1'],
                                    'mora':['Mora 4','Mora 3','Mora 2','Mora 1','Al día','Pagado']})

fig.update_layout(height = 600, width=1200, bargap=0.1)

fig.show()

In [57]:
cashflow.sample(10)

,id_amortizacion,id_credito,num_cuota,saldo_total,monto_cuota,saldo_exigible,estado,fecha_cuota,fecha_pago,monto_seguro,...,exigible_moneda,tasa,numero_periodos,score_hit,score_no_hit,valor_financiamiento,prima,accesorios,cuotas_pendientes,mora
20276,21038,HN0001408,8,21.85,21.85,NaN,Fijo,2025-01-14,NaN,NaN,...,NaN,3.50,8,0,719,2400.0,600.0,0.0,1.0,Al día
5161,5647,SV0000140,9,79.83,19.96,NaN,Fijo,2025-01-05,NaN,NaN,...,NaN,4.15,12,0,505,96.0,24.0,0.0,0.0,Al día
27140,28014,SV0000766,16,14.05,14.05,NaN,Fijo,2025-05-28,NaN,NaN,...,NaN,4.15,16,0,580,112.8,28.2,0.0,NaN,Al día
24598,25416,HN0001687,14,53.05,17.68,NaN,Fijo,2025-04-22,NaN,NaN,...,NaN,3.50,16,0,713,2640.0,660.0,0.0,0.0,Al día
28543,29425,HN0001924,11,44.34,22.16,NaN,Fijo,2025-03-15,NaN,NaN,...,NaN,4.30,12,0,416,2600.0,1400.0,0.0,NaN,Al día
6268,6818,SV0000168,8,29.84,29.84,NaN,Fijo,2024-12-23,NaN,NaN,...,NaN,4.15,8,0,628,120.0,30.0,0.0,0.0,Al día
11579,12173,HN0000842,15,55.67,27.83,NaN,Fijo,2025-04-16,NaN,NaN,...,NaN,4.15,16,0,532,3650.0,1350.0,500.0,0.0,Al día
28054,28936,SV0000806,18,113.25,17.00,NaN,Fijo,2025-06-28,NaN,NaN,...,NaN,3.50,24,0,741,109.6,27.4,0.0,NaN,Al día
6646,7196,HN0000553,6,126.85,42.28,NaN,Fijo,2024-11-25,NaN,NaN,...,NaN,4.15,8,0,495,4250.0,1297.0,0.0,1.0,Al día
13312,13934,SV0000352,4,91.03,18.21,NaN,Fijo,2024-11-04,NaN,NaN,...,NaN,3.50,8,0,744,80.0,20.0,0.0,0.0,Al día


In [78]:
historico = cashflow[~cashflow['estado'].isin(['Fijo','Exigible'])]
historico['fecha_cuota'] = pd.to_datetime(historico['fecha_cuota'])
historico['pagado'] = historico['monto_cuota'].where(historico['estado']=='Pagado',0)

In [87]:
hdf = historico.groupby(['estado','fecha_cuota']).agg({'id_credito':'count','fecha_pago':'count','monto_cuota':'sum','pagado':'sum'}).reset_index()

In [92]:
hist_viz = historico.set_index('fecha_cuota').resample('W').agg({'id_credito':'count','fecha_pago':'count','monto_cuota':'sum','pagado':'sum','estado':'count'})

In [94]:
hist_viz['impago'] = hist_viz['id_credito'] - hist_viz['fecha_pago']
hist_viz['vencido'] = hist_viz['monto_cuota'] - hist_viz['pagado']

In [113]:
# Cuotas impagos cantidad 
fig = px.histogram(historico, template='simple_white', x=historico.fecha_cuota, histfunc='sum', color='mora', barnorm='percent',
                   color_discrete_map={'Mora 4':'black','Mora 3':'purple','Mora 2':'purple','Mora 1':'violet', 'Pagado':'green','Al día':'lightblue'},
                   category_orders={'quincena':['4','3','2','1'],
                                    'mora':['Mora 4','Mora 3','Mora 2','Mora 1','Al día','Pagado']})

fig.show()

In [ ]:
# Suma cuotas impagos monto

In [95]:
hist_viz

,id_credito,fecha_pago,monto_cuota,pagado,estado,impago,vencido
fecha_cuota,,,,,,,
2024-08-18,3,2,70.99,47.65,3,1,23.34
2024-08-25,53,39,1592.44,1219.96,53,14,372.48
2024-09-01,117,74,3335.89,2142.66,117,43,1193.23
2024-09-08,393,246,11457.43,7062.85,393,147,4394.58
2024-09-15,386,254,10958.56,7245.03,386,132,3713.53
2024-09-22,903,447,26188.94,12554.63,903,456,13634.31
2024-09-29,718,401,20708.50,11496.33,718,317,9212.17
2024-10-06,447,257,12270.67,7004.45,447,190,5266.22


In [98]:
fig = go.Figure(data=[
	
	go.Bar(name='Pagos', 
		   x=hist_viz.index, 
		   y=hist_viz.fecha_pago.cumsum(),
		   marker=dict(color='green'),
		   opacity=0.6,
		   text=hist_viz.pagado.cumsum(),
		   texttemplate='$%{text:,.2f}',
		   textposition='inside'),
	
	go.Bar(name='Impagos', 
		   x=hist_viz.index, 
		   y=hist_viz.impago.cumsum(),
		   marker=dict(
			   color='indigo',
			   #colorscale='Reds',
			   #colorbar=dict(title="Value")
		   ),
		   #opacity=0.6,
		   text=hist_viz.vencido.cumsum(),
		   texttemplate='$%{text:,.2f}',
		   textposition='inside'),
	
	#go.Scatter(name='Límite 25% impago',
	#		   x=df_t.index,
	#		   y=df_t.meta_cantidad,
	#		   mode='lines+markers',
	#		   marker=dict(color='yellow'),)
])

fig.update_layout(title='Pagos acumulados', template='simple_white', barmode='stack',height=600, width=1200)
fig.show()

In [ ]:
# Tasa de impago acumulada actual

#### **Timeseries**

Bar, stacked bar
Line, area

In [ ]:
# Impagos/Due cantidad normalizado

In [105]:
fig = go.Figure(layout=go.Layout(
	title="Ingresos acumulados",
	width=1000,
	height=600,
	template='simple_white'
))

fig.add_trace(go.Scatter(x=hist_viz.index,
						 y=hist_viz.monto_cuota.cumsum(),
						 mode='lines',
						 line=dict(width=0.5, color='purple'),
						 fill='tozeroy',
						 name='Ingreso esperado acumulado'))

fig.add_trace(go.Scatter(x=hist_viz.index,
						 y=hist_viz.pagado.cumsum(),
						 mode='lines',
						 line=dict(width=0.5, color='green'),
						 fill='tozeroy',
                         fillcolor='green',
						 name='Ingreso real acumulado'))

fig.add_trace(go.Scatter(x=hist_viz.index, y=pd.Series(hist_viz['monto_cuota']*0.75).cumsum(), mode='lines', line=dict(width=1.5, color='black', dash='dash'), name='Meta 75%'))

fig.show()

In [ ]:
filter

In [ ]:
# Impagos/Due monto total

In [ ]:
# Monto esperado/recibido acumulado

### **Composition**

#### **Cohort analysis**

In [ ]:
# Reporte de mora por cosecha

In [ ]:
# Vintage analysis

In [117]:
fig = px.histogram(cashflow, template='simple_white',
                   x='fecha_cuota',
                   y='monto_cuota',
                   color='estado')

fig.show()

### **Comparison**

#### **Ventas**

In [ ]:
# Top por categoría

#### **Moras**

In [ ]:
# Top contribuyentes

In [ ]:
# Mapa impago